In [1]:
using rhd2000evalboard

In [1]:
const mylib="/home/nicolelislab/neural-analysis-toolbox/DataAcq/libokFrontPanel.so"

"/home/nicolelislab/neural-analysis-toolbox/DataAcq/libokFrontPanel.so"

In [2]:
#Opal Kelly XEM6010 board
x=open_board()

Constructed
Found 1 Opal Kelly device(s)
Serial number is 141100085D


Ptr{Void} @0x00000000040947b0

In [3]:
# Load Rhythm FPGA configuration bitfile (provided by Intan Technologies).
uploadFpgaBitfile();

0


In [4]:
#Initialize board
initialize_board()

LoadError: type cannot be constructed
while loading In[4], in expression starting on line 2

In [3]:
setDataSource(0,0)

Found 0 Opal Kelly device(s)


In [3]:
#Select per-channel amplifier sampling rate
setSampleRate(20000)

"PortA"

In [4]:
#Now that we have set our sampling rate, we can set the MISO sampling delay
#which is dependent on the sample rate. We assume a 3-foot cable.
setCableLengthFeet("PortA", 3.0)

true

In [1]:
# Let's turn one LED on to indicate that the program is running.
ledArray=[1,0,0,0,0,0,0,0]
setLedDisplay(ledArray)

LoadError: setLedDisplay not defined
while loading In[1], in expression starting on line 3

In [2]:
#Set up RHD2000 register object using this sample rate to optimize MUX-
#related register settings

In [ ]:
#Set to run continuously
setContinuousRunMode(true)
run()

In [ ]:
#continuously sample
usbDataRead=true
while (usbDataRead || isRunning())
    #If Fifo contains words, they are read
    usbDataRead = readDataBlocks(1,dataQueue)
    if size(dataQueue.size() > = 50)
        queueToFile(dataQueue,saveOut)
    end
end

queueToFile(dataQueue,saveOut)

flush()